In [103]:
import numpy as np
import math
import time

f = open("Input.dat")
num_messages = int(f.readline())
tau = float(f.readline())

priorities = np.zeros(num_messages)
transmission_times = np.zeros(num_messages)
periods = np.zeros(num_messages)

for idx, parameters in enumerate(f.readlines()):
    parameters = parameters.split()
    priorities[idx] = int(parameters[0])
    transmission_times[idx] = float(parameters[1])
    periods[idx] = int(parameters[2])

priorities = np.array([13.  6.  1.  5.  0.  3.  9.  2.  7.  8. 12. 14.  4. 16. 15. 11. 10])

In [104]:
def getResponses(priorities, transmission_times, periods):
    i = 0
    first_time = True
    responses = []
    while i < num_messages:  

        # Initialize
        # B = max(transmission_times[i:])
        B = max(transmission_times[priorities >= priorities[i]])

        if first_time:
            Q = B

        RHS = B
        for index in range(len(priorities)):
            if priorities[index] < priorities[i]:
                RHS += math.ceil((Q+tau)/periods[index]) * transmission_times[index]

        # check constraint
        if RHS + transmission_times[i] > periods[i]:
            # print(f"Need to reschedule")
            responses.append(-1)
            i += 1
            continue


        if Q == RHS:
            # print(f"R_{i} = {RHS+transmission_times[i]}")
            responses.append(RHS+transmission_times[i])
            i += 1
            first_time = True
        else:
            Q = RHS
            first_time = False
            
    return responses

In [105]:
res = getResponses(priorities, transmission_times, periods)
print(res.count(-1))
print(res)
print(np.sum(getResponses(priorities, transmission_times, periods)))

0
[1.44, 2.04, 2.56, 3.16, 3.68, 4.28, 5.2, 8.4, 9.0, 9.68, 10.2, 19.36, 19.8, 20.32, 29.400000000000002, 29.759999999999998, 30.279999999999998]
208.55999999999997


# Stimulated Annealing

In [106]:
np.random.seed(int(time.time()))
temperature = 50
r = 0.99
solution_star = priorities[:]
previous_responses = getResponses(priorities, transmission_times, periods)
previous_obj = np.sum(previous_responses) + (previous_responses.count(-1) * 501)
best_obj = previous_obj
best_priorities = np.zeros(len(priorities))

while temperature >= 1:
    
    # pick a random neighbor S' of S
    [i, j] = np.random.randint(priorities.shape[0], size=2)
    priorities[i], priorities[j] = priorities[j], priorities[i]
    
    # delta_c = cost(S') - cost(S)
    responses = getResponses(priorities, transmission_times, periods)
    c = responses.count(-1)
    worst = np.sum(responses)
    objective = worst + ( c * 501 ) 
    delta_c = objective - previous_obj
    
    # if    delta_c <  0, then S = S'
    # elif  delta_c >= 0, then S = S' with probability e–∆C/T
    probability = math.exp((-1 * delta_c) / temperature)
    rnd = np.random.rand()
    
    if objective < best_obj:
        best_obj = objective
        best_priorities = priorities.copy()
    
    
    if delta_c >= 0 and ((rnd - probability) > 0):
        priorities[i], priorities[j] = priorities[j], priorities[i]

#     print(f"T : {temperature}")
#     print(f"responses :  {responses}")
#     print(f"objective : {objective}")
#     print(f"best objective : {best_obj}")›
#     print(f"delta_c : {delta_c}")
#     print(f"rnd : {rnd}")
#     print(f"probability : {probability}")
#     print(f"priorities : {priorities}")
#     print("")

        
    # T = rT
    temperature *= r
    
print(best_obj)

204.35999999999999


In [107]:
print(best_priorities)

[13.  6.  1.  5.  0.  3.  9.  2.  7.  8. 12. 14.  4. 16. 15. 11. 10.]


In [38]:
a = [3, 4, 5, 6, 7]
a[0], a[1] = a[1], a[0]
print(a)

[4, 3, 5, 6, 7]
